In [1]:
import pickle
import json
import sys
import os
import string
import requests
import ast  # for string to list: ast.literal_eval()
from bs4 import BeautifulSoup
import bs4
import time
import matplotlib.pyplot as plt
from datetime import datetime
from datetime import date
import pandas as pd
import numpy as np
import re
from tqdm.notebook import tqdm
import warnings
data_dir = "/Users/vigadam/Documents/github/media_network/media_data/html_pages/888"

In [186]:
class Helper:
    import json

    from bs4 import BeautifulSoup
    import bs4

    import pandas as pd
    import numpy as np

    import re

    import warnings

    warnings.filterwarnings("ignore")

    def __init__(self, df):
        None

    def title_main(self, element, _id):
        for i, soup in tqdm(enumerate(self.soups)):
            try:
                df.loc[self.soups.index[i], "Title"] = (
                    BeautifulSoup(soup).find(attrs={element: _id}).get_text().strip()
                )
            except AttributeError:
                df.loc[self.soups.index[i], "Title"] = None

    def link(self):
        for i, soup in tqdm(enumerate(self.soups)):
            soup_for_links = BeautifulSoup(soup).find_all("a")
            link_list = []
            for item in soup_for_links:
                if type(item.get("href")) == str:
                    link_list.append(item.get("href"))
            df.loc[self.soups.index[i], "links"] = str(list(set(link_list)))

    def text_main(self, element, class_id):
        for i, soup in tqdm(enumerate(self.soups)):
            try:
                soup_for_text = (
                    BeautifulSoup(soup)
                    .find(element, class_=re.compile(class_id))
                    .find_all("p")
                )
                text_list = []
                for item in soup_for_text:
                    if soup_for_text != "":
                        text_list.append(item.text)
                df.loc[self.soups.index[i], "Text"] = " ".join(text_list)
            except AttributeError:
                df.loc[self.soups.index[i], "Text"] = None

    def author_main(self, element, _id):
        for i, soup in tqdm(enumerate(self.soups)):
            try:
                df.loc[self.soups.index[i], "Author"] = (
                    BeautifulSoup(soup).find(attrs={element: _id}).get_text().strip()
                )
            except:
                df.loc[self.soups.index[i], "Author"] = None

    def category_main(self, element, id_):
        for i, soup in tqdm(enumerate(self.soups)):
            try:
                df.loc[self.soups.index[i], "Category"] = (
                    BeautifulSoup(soup).find(attrs={element: id_}).text.strip()
                )

            except:
                df.loc[self.soups.index[i], "Category"] = None

    def tags_main(self, element, id_):
        for i, soup in tqdm(enumerate(self.soups)):
            try:
                df.loc[self.soups.index[i], "Tags"] = (
                    BeautifulSoup(soup).find(attrs={element: id_}).text.strip()
                )

            except:
                df.loc[self.soups.index[i], "Tags"] = None
        df["Tags"] = df["Tags"].str.split("\n\n\n")

    def clean_(self):
        self.title()
        self.text()
        self.link()
        self.author()
        self.source()
        self.tags()

In [248]:
class _888(Helper):
    def __init__(self, df):
        self.data = df
        self.soups = self.data["soup"].dropna()

    def clean(self):
        for i, Soup in tqdm(enumerate(self.soups)):

            soup = BeautifulSoup(Soup)
            
            #Title
            try:
                df.loc[self.soups.index[i], "Title"] = (
                    soup.find("h1").get_text().strip()
                )
            except:
                df.loc[self.soups.index[i], "Title"] = None
            
            #Text
            try:
                textlist = [soup.find(attrs={"class": "high-lite"})] + soup.find(
                    attrs={"class": "maincontent8"}
                ).find_all("p")
                df.loc[self.soups.index[i], "Text"] = (
                    " ".join([item.get_text().strip() for item in textlist])
                    .replace("\xa0", "")
                    .strip()
                )
            except:
                df.loc[self.soups.index[i], "Text"] = None
            
            #Author
            try:
                df.loc[self.soups.index[i], "Author"] = (
                    soup.find(attrs={"class": "note-block"}).get_text().strip()
                )
            except:
                df.loc[self.soups.index[i], "Author"] = None
            
            #date
            try:
                df.loc[self.soups.index[i], "date"] = (
                    soup.find(attrs={"id": "cikkholder"})
                    .find("p")
                    .get_text()
                    .split(" ")[0][:-1]
                    .replace(".", "-")
                )
            except:
                df.loc[self.soups.index[i], "date"] = None
            
            #tags
            try:
                taglist = soup.find(attrs={"class": "plugin-holder"}).find_all(
                    attrs={"rel": "tag"}
                )
                df.loc[self.soups.index[i], "Tags"] = str(
                    [item.get_text().strip().lower() for item in taglist]
                )
            except:
                df.loc[self.soups.index[i], "Tags"] = None

In [199]:
data_files = os.listdir(data_dir + "/all")

data_files = [data_dir + "/all/" + item for item in data_files]

In [241]:
df_missed = pd.DataFrame()
for data_file in tqdm(data_files):

    df = pd.read_pickle(data_file)

    df_missed = pd.concat(
        [df_missed, df.loc[df["soup"] == "404", ["link","rovat"]].reset_index(drop=True)]
    )

In [247]:
df_missed.reset_index(drop=True).to_csv(
    "/Users/vigadam/Documents/github/media_network/media_data/links_to_sites/888_missed.csv"
)

In [249]:
df_clean = pd.DataFrame()
for data_file in tqdm(data_files):

    df = pd.read_pickle(data_file)
    df = df.loc[df["soup"] != "404"]

    _888(df).clean()

    df_clean = pd.concat([df_clean, df.drop(columns=["Unnamed: 0", "soup"])])

KeyboardInterrupt: 

In [283]:
df = pd.read_pickle(
    "/Users/vigadam/Dropbox/My Mac (MacBook-Air.local)/Documents/github/media_network/media_data/clean_text/pickle_format/888_clean_all.pkl"
).drop(columns=["Unnamed: 0.1","felosztas"])

In [284]:
df = df.dropna(subset=["date"])

In [285]:
df["year"] = df["date"].str.split("-").apply(lambda x: int(x[0]))

In [288]:
df_2020 = df.loc[df["year"] == 2020].drop(columns=["year"])

In [291]:
df_2020.to_csv("/Users/vigadam/Dropbox/My Mac (MacBook-Air.local)/Documents/github/media_network/media_data/clean_text/csv_format/2020/888_clean.csv")

In [275]:
df["date"].isnull().sum()

3

In [293]:
data_dir = "/Users/vigadam/Dropbox/My Mac (MacBook-Air.local)/Documents/github/media_network/media_data/clean_text/csv_format/2020/"

In [294]:
covid = pd.read_csv("/Users/vigadam/Dropbox/My Mac (MacBook-Air.local)/Documents/github/media_network/media_data/covid data/owid-covid-data.csv")

In [296]:
covid.columns

Index(['iso_code', 'continent', 'location', 'date', 'total_cases', 'new_cases',
       'new_cases_smoothed', 'total_deaths', 'new_deaths',
       'new_deaths_smoothed', 'total_cases_per_million',
       'new_cases_per_million', 'new_cases_smoothed_per_million',
       'total_deaths_per_million', 'new_deaths_per_million',
       'new_deaths_smoothed_per_million', 'reproduction_rate', 'icu_patients',
       'icu_patients_per_million', 'hosp_patients',
       'hosp_patients_per_million', 'weekly_icu_admissions',
       'weekly_icu_admissions_per_million', 'weekly_hosp_admissions',
       'weekly_hosp_admissions_per_million', 'total_tests', 'new_tests',
       'total_tests_per_thousand', 'new_tests_per_thousand',
       'new_tests_smoothed', 'new_tests_smoothed_per_thousand',
       'positive_rate', 'tests_per_case', 'tests_units', 'total_vaccinations',
       'new_vaccinations', 'new_vaccinations_smoothed',
       'total_vaccinations_per_hundred',
       'new_vaccinations_smoothed_per_mil

In [295]:
covid.groupby("iso_code")["iso_"]

,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,gdp_per_capita,extreme_poverty,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index
0,AFG,Asia,Afghanistan,2020-02-24,1.0,1.0,NaN,NaN,NaN,NaN,...,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.498
1,AFG,Asia,Afghanistan,2020-02-25,1.0,0.0,NaN,NaN,NaN,NaN,...,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.498
2,AFG,Asia,Afghanistan,2020-02-26,1.0,0.0,NaN,NaN,NaN,NaN,...,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.498
3,AFG,Asia,Afghanistan,2020-02-27,1.0,0.0,NaN,NaN,NaN,NaN,...,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.498
4,AFG,Asia,Afghanistan,2020-02-28,1.0,0.0,NaN,NaN,NaN,NaN,...,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.498
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60703,ZWE,Africa,Zimbabwe,2021-01-13,24256.0,1017.0,921.714,589.0,38.0,22.571,...,1899.775,21.4,307.846,1.82,1.6,30.7,36.791,1.7,61.49,0.535
60704,ZWE,Africa,Zimbabwe,2021-01-14,25368.0,1112.0,956.143,636.0,47.0,27.143,...,1899.775,21.4,307.846,1.82,1.6,30.7,36.791,1.7,61.49,0.535
60705,ZWE,Africa,Zimbabwe,2021-01-15,26109.0,741.0,921.286,666.0,30.0,28.286,...,1899.775,21.4,307.846,1.82,1.6,30.7,36.791,1.7,61.49,0.535
60706,ZWE,Africa,Zimbabwe,2021-01-16,26881.0,772.0,911.714,683.0,17.0,28.571,...,1899.775,21.4,307.846,1.82,1.6,30.7,36.791,1.7,61.49,0.535
